# Pre-requisites

General imports and model setup.

In [39]:
import re
import os.path
from langchain_core.tools import tool, Tool
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser

# This is just to avoid annoying DuckDuckGo warnings
import warnings
warnings.filterwarnings("ignore")

@tool
def extricate(s: str) -> str:
  """Extricates a string
     Args:
       s: The string to extricate
  """
  return s.lower().replace('o', '0')[::-1]

SERPER_API_KEY = os.environ.get("SERPER_API_KEY")
if not SERPER_API_KEY:
  if os.path.exists('SERPER_API_KEY'):
    with open('SERPER_API_KEY') as f:
      os.environ['SERPER_API_KEY'] = f.read().strip()
  else:
    raise ValueError('Please provide SERPER_API_KEY either as an env varible or inside a file name')

search_func = GoogleSerperAPIWrapper()
search = Tool(
  name='search',
  func=search_func.run,
  description='use this when you need to search for new information on the web'
)

tools = [extricate, search_tool]
tools_dict = {t.name: t for t in tools}

#MODEL='qwen3:14b'
MODEL='qwen3:32b'
llm = ChatOllama(model=MODEL).bind_tools(tools)

def clean_response(response):
    return re.sub(r"<think>.*?</think>\n?", "", response, flags=re.DOTALL)

# Testing the DuckDuckGo tool

In [18]:
search.invoke("Levante - Barcelona")

"Watch the highlights as Barcelona overcomes a 2-0 deficit to defeat Levante, 3-2 , in their LALIGA matchup. The game-winning goal was scored ... Barcelona came from two goals down to beat Levante 3-2 and make it successive wins to start their season. Levante frustrated the champions ... Despite trailing 2-0 at half-time, FC Barcelona collect three points in Valencia thanks to second half goals from Pedri, Ferran and an injury ... La Liga release VAR tape explaining why Barcelona conceded controversial penalty on way to comeback win at Levante · Barca concede controversial ... Levante 2-3 FC Barcelona: Still the comeback kings. Despite trailing 2-0 at half-time, the Catalans collect three points in Valencia thanks to ... Barcelona came from two goals down to pull off a monumental comeback to defeat Levante 3–2 and capture three points that could prove ... Barcelona came from two goals down to win 3-2 draw at newly-promoted Levante in Saturday evening's see-saw La Liga clash — with the 

# Running the tool manually from an LLM

In [44]:
messages = [HumanMessage("Take the name of the scorers in the last Levante - Barcelona soccer match and extricate them")]
result = llm.invoke(messages)
messages.append(result)

while result.tool_calls:
  for tool_call in result.tool_calls:
    print(f"Running {tool_call['name']} ({tool_call['args']})...")
    tool_output = tools_dict[tool_call['name']].invoke(tool_call)
    messages.append(tool_output)
  
  result = llm.invoke(messages)
  messages.append(result)

for m in messages:
  m.pretty_print()

print("\n*** FINAL RESPONSE ***\n")
print(clean_response(messages[-1].content))

Running search ({'__arg1': 'last Levante vs Barcelona soccer match scorers'})...
Running extricate ({'s': 'Ivan Romero, Pedri, Ferran Torres, Unai Elgezabal'})...
================================ Human Message =================================

Take the name of the scorers in the last Levante - Barcelona soccer match and extricate them
================================== Ai Message ==================================

<think>
Okay, the user wants the names of the scorers from the last Levante vs. Barcelona match and then to extricate them. First, I need to find out who the scorers were. Since I don't have real-time data, I should use the search function to look up the latest match between these two teams. Let me check the date of the most recent game. Once I have the match date and result, I can identify the players who scored. After retrieving the names, the extricate function will process them. But wait, what does "extricate" mean here? The function's description says it extricates a s

# Running as an agent

In [43]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

q = (
  "What's the latest film starring Penelope Cruz's spouse?"
)

input_messages = [
  {
    'role': 'system',
    'content': (
      'You are an expert at looking up and synthesising information '
      'to reply to questions. Think before you launch any search queries '
      'and try to structure them in a way that can be understood by a '
      'traditional search engine. Divide the question into different queries, '
      'each with its own keywords, if necessary.'
    )
  },
  {
    'role': 'user',
    'content': q
  }
]

seen_message_ids = set()
agent_messages = []
for step in agent.stream({'messages': input_messages}, stream_mode="values"):
  for message in step['messages']:
    if id(message) not in seen_message_ids:
      message.pretty_print()
      seen_message_ids.add(id(message))
  agent_messages = step['messages']

print("\n*** FINAL RESPONSE ***\n")
print(clean_response(agent_messages[-1].content))

================================ System Message ================================

You are an expert are looking up and synthesising information to reply to questions. Think before you launch any search queries and try to structure them in a way that can be understood by a traditional search engine. Divide the question into different queries, each with its own keywords, if necessary.
================================ Human Message =================================

What's the latest film starring Penelope Cruz's spouse?
================================== Ai Message ==================================

<think>
Okay, let's see. The user is asking for the latest film starring Penelope Cruz's spouse. First, I need to figure out who Penelope Cruz's spouse is. I know she's a famous actress, so maybe her husband is also in the entertainment industry.

I should start by searching for Penelope Cruz's spouse. Let me use the search function. The query would be "Penelope Cruz spouse". That should giv